# Create a time-of-flight lookup table for BIFROST

In [ ]:
from ess.reduce import nexus
import sciline
import scipp as sc
import scippnexus as snx
from ess.reduce.nexus.types import  RawChoppers, DiskChoppers
from ess.reduce.time_of_flight import TofLutProvider
from ess.reduce.time_of_flight.types import *
from scippneutron.chopper import DiskChopper

from ess.bifrost import BifrostSimulationWorkflow
from ess.bifrost.data import simulated_elastic_incoherent_with_phonon
from ess.bifrost.types import *
from ess.spectroscopy.types import *

## Prepare the workflow

The choppers in the simulated file need to be processed before they can be used for computing a lookup table.
The following functions work for the specific simulation but are *not* usable in general.

In [ ]:
def extract_chopper_plateau(chopper):
    processed = chopper.copy()
    # These are constant in the simulated data.
    processed['rotation_speed'] = processed['rotation_speed'].data.mean()
    processed['phase'] = processed['phase'].data.mean()
    # Guessing here as this is not stored in the file.
    processed['beam_position'] = sc.scalar(0.0, unit='deg')
    return DiskChopper.from_nexus(processed)


def extract_chopper_plateaus(choppers: RawChoppers[RunType]) -> DiskChoppers[RunType]:
    return DiskChoppers[RunType](choppers.apply(extract_chopper_plateau))

Construct the workflow.
We only need the detector names for the geometry up to the sample, so choosing a single detector is enough.

In [ ]:
with snx.File(simulated_elastic_incoherent_with_phonon()) as f:
    detector_names = list(f['entry/instrument'][snx.NXdetector])

In [ ]:
workflow = BifrostSimulationWorkflow(detector_names=detector_names[:1],
                                     tof_lut_provider=TofLutProvider.TOF)
workflow.insert(extract_chopper_plateaus)
workflow[Filename[SampleRun]] = simulated_elastic_incoherent_with_phonon()

Compute the required distance range:

In [ ]:
beamline = sciline.compute_mapped(workflow, BeamlineWithSpectrometerCoords[SampleRun])[0]
monitor = workflow.compute(nexus.types.CalibratedMonitor[SampleRun, FrameMonitor3])

In [ ]:
l_monitor = sc.norm(monitor.coords['source_position'] - monitor.coords['position'])
l_min = l_monitor
l_max = beamline.coords['L1']
workflow[NumberOfSimulatedNeutrons] = 5_000_000
workflow[L1Range] = (l_min, l_max)

## Compute the lookup table

In [ ]:
workflow.visualize(TimeOfFlightLookupTable, graph_attr={"rankdir": "LR"})

In [ ]:
table = workflow.compute(TimeOfFlightLookupTable)
table

In [ ]:
table.squeeze().plot()

## Save to file

In [ ]:
table.save_hdf5('BIFROST-simulation-tof-lookup-table.h5')